<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise



In [11]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=c36c74edf24173f74296e023231d5d4129e9a7801e1835e5eb79ff744537b63f
  Stored in directory: /root/.cache/pip/wheels/a3/6b/89/3bec14288af8e0729d088fb41a785e583d49e772f2529c7676
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16224 sha256=76f6ee7f680a4af135fa73e9344afb6ba4865955d4696b40120be68441fa3970
  Stored in directory: /root/.cache/pip/wheels/fc/40/7a/4d3b2dc2e80b4b3bbda89eec94fcefd6c8bd1101cc7bb89554
  Created wheel for stdiomask: filename=stdiomask

In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# I added two more codes to enhance the visualisation
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response
# Typing support for cleaner, testable function signatures
from typing import Union, List, Dict

## 🌤️ Weather Data Functions

In [3]:
# Define get_weather_data() function here
from typing import Union, List, Dict

def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    Fetch weather data for one or more locations and return it as a dictionary.

    Args:
        location (str or list): A single location name or a list of locations.
        forecast_days (int): Number of forecast days to retrieve (default is 5).

    Returns:
        dict: A dictionary mapping each location to its weather data or error.
              Format:
              {
                  "Paris": {
                      "current_conditions": {...},
                      "forecast": [{...}, {...}, ...]
                  },
                  "InvalidCity": {
                      "error": "Could not retrieve weather data"
                  }
              }
    """
    if isinstance(location, str):
        location = [location]  # Convert single string to list

    results = {}

    for loc in location:
        try:
            weather_response = get_weather(loc)  # Assumed to return a WeatherResponse object
            if not hasattr(weather_response, 'dict'):
                raise TypeError("Weather object does not support dict() conversion.")

            weather_data = weather_response.dict()

            # Safe access to forecast and current conditions
            forecast = weather_data.get("forecast", [])
            current = weather_data.get("current_conditions", {})

            if not forecast:
                raise ValueError("Forecast data is missing or empty.")

            # Limit forecast to specified number of days
            results[loc] = {
                "current_conditions": current or "No current data available",
                "forecast": forecast[:forecast_days]
            }

        except Exception as e:
            results[loc] = {"error": f"Failed to retrieve weather for {loc}: {e}"}

    return results



❌ Failed to fetch weather data: name 'get_weather' is not defined


## 📊 Visualisation Functions

In [4]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
import matplotlib.pyplot as plt

def create_temperature_visualisation(weather_data, output_type='display'):
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    for location, times_data in weather_data.items():
        times = list(times_data.keys())
        temps = [times_data[time]['temperature'] for time in times]
        ax.plot(times, temps, marker='o', label=location)

        # Optional: annotate points
        for i, temp in enumerate(temps):
            ax.annotate(f"{temp}°", (times[i], temps[i]), textcoords="offset points", xytext=(0,5), ha='center', fontsize=8)

    ax.set(title='Temperature Forecast', xlabel='Time Period', ylabel='Temperature (°C)')
    ax.legend()
    ax.grid(True)
    plt.xticks(rotation=45)

    if output_type == 'figure':
        return fig
    plt.show()




In [ ]:
import matplotlib.pyplot as plt

def create_precipitation_visualisation(weather_data, output_type='display'):
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    width = 0.35  # Width of the bars
    locations = list(weather_data.keys())
    time_labels = list(next(iter(weather_data.values())).keys())
    x = range(len(time_labels))

    for idx, location in enumerate(locations):
        times_data = weather_data[location]
        precip_values = [times_data[time]['precipitation'] for time in time_labels]
        offsets = [i + (idx * width) for i in x]
        ax.bar(offsets, precip_values, width=width, label=location)

        # annotate bars
        for i, value in enumerate(precip_values):
            ax.text(offsets[i], value + 1, f"{value}%", ha='center', fontsize=8)

    ax.set(
        title="Chance of Rain by Location",
        xlabel="Time Period",
        ylabel="Chance of Rain (%)"
    )
    ax.set_xticks([i + width/2 for i in x])
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(True)

    if output_type == 'figure':
        return fig
    plt.show()



## 🤖 Natural Language Processing

In [13]:
# Define parse_weather_question() and generate_weather_response() here

import re

def parse_weather_question(question):

    question = question.strip()

    # Extract location: match capitalised word(s) after 'in' or end of question
    loc_match = re.search(r'in ([A-Z][a-z]+(?: [A-Z][a-z]+)*)', question)
    location = loc_match.group(1) if loc_match else 'your area'

    # Time detection with more flexibility
    if 'tomorrow' in question.lower():
        time = 'tomorrow'
    elif 'weekend' in question.lower():
        time = 'this weekend'
    elif re.search(r'on (Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)', question, re.IGNORECASE):
        time = 'that day'
    else:
        time = 'today'

    # Detect attribute
    attribute = 'general'
    for attr in ['temperature', 'rain', 'snow', 'wind', 'humidity']:
        if attr in question.lower():
            attribute = attr
            break

    return {'location': location, 'time_period': time, 'attribute': attribute}

def generate_weather_response(parsed, data):
    loc = parsed['location']
    time = parsed['time_period']
    attr = parsed['attribute']

    if loc not in data or time not in data[loc]:
        return f"❌ Sorry, no forecast available for {loc} {time}."

    forecast = data[loc][time]

    # Handle each attribute
    if attr == 'temperature':
        return f"🌡️ Temperature in {loc} {time} is {forecast.get('temperature', 'unknown')}°C."
    elif attr == 'rain':
        rain = '🌧️ rain' if forecast.get('rain') else '☀️ no rain'
        return f"Expected: {rain} in {loc} {time}."
    elif attr == 'snow':
        snow = '❄️ snow' if forecast.get('snow') else 'no snow'
        return f"Expected: {snow} in {loc} {time}."
    elif attr == 'wind':
        return f"💨 Wind speed in {loc} {time} is {forecast.get('wind', 'unknown')} km/h."
    elif attr == 'humidity':
        return f"💧 Humidity in {loc} {time} is {forecast.get('humidity', 'unknown')}%."
    else:
        # General response
        temp = forecast.get('temperature', 'unknown')
        rain = 'rain' if forecast.get('rain') else 'no rain'
        return f"In {loc} {time}, temperature is {temp}°C with {rain}."



## 🧭 User Interface

In [14]:
# Define menu functions using pyinputplus or ipywidgets here

import pyinputplus as pyip

def main_menu():
    print("\n=== 🌦️ Weather Assistant Menu ===")
    return pyip.inputMenu(
        ['Ask Weather Question', 'Visualize Weather Data', 'Exit'],
        numbered=True,
        prompt="Select an option:\n",
        blank=False,
        limit=3,
        retries=2,
        default='Exit'
    )

def visualization_menu():
    return pyip.inputMenu(
        ['Temperature', 'Precipitation', 'Back to Main Menu'],
        numbered=True,
        prompt="Choose a visualization type:\n",
        blank=False,
        limit=3,
        retries=2,
        default='Back to Main Menu'
    )

def run_interface():
    while True:
        choice = main_menu()

        if choice == 'Ask Weather Question':
            # Call weather question function here
            print("👉 You chose to ask a weather question.")

        elif choice == 'Visualize Weather Data':
            while True:
                sub_choice = visualization_menu()
                if sub_choice == 'Temperature':
                    print("📈 Showing temperature chart...")
                elif sub_choice == 'Precipitation':
                    print("🌧️ Showing precipitation chart...")
                elif sub_choice == 'Back to Main Menu':
                    break

        elif choice == 'Exit':
            print("👋 Exiting Weather Assistant. Stay safe!")
            break

return pyip.inputMenu(
    ['(A) Ask Weather Question', '(V) Visualize Weather Data', '(E) Exit'],
    numbered=False,
    lettered=True
)


## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

#🌤️ Weather Data Functions

#Sample 1; My code.
def get_weather_data(location, forecast_days=5):
    try:
        weather_obj = get_weather(location)
        data = weather_obj.dict()
        if "forecast" in data:
            data["forecast"] = data["forecast"][:forecast_days]

        return data

    except Exception as e:
        return {"error": str(e)}

test_location = "Sydney"
weather = get_weather_data(test_location)

if "error" in weather:
    print(f"Failed to fetch weather data: {weather['error']}")
else:
    print(f"Current weather in {test_location}:")
    print(weather.get("current_conditions", "No current conditions available."))
    print("\nForecast:")
    for day in weather.get("forecast", []):
        print(day)

#Sample 2; improved version provided by Chatgbt.
def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    loc = parsed['location']
    time = parsed['time_period']
    attr = parsed['attribute']

    if loc not in data or time not in data[loc]:
        return f"❌ No forecast available for {loc} {time}."

    forecast = data[loc][time]

    if attr == 'temperature':
        temp_c = forecast.get('temperature', 'unknown')
        if isinstance(temp_c, (int, float)):
            temp = convert_temperature(temp_c, temp_unit)
            unit_symbol = '°F' if temp_unit == 'Fahrenheit' else '°C'
            return f"🌡️ Temperature in {loc} {time} is {temp}{unit_symbol}."
        return "Temperature data unavailable."

    elif attr == 'wind':
        wind_kmh = forecast.get('wind', 'unknown')
        if isinstance(wind_kmh, (int, float)):
            wind = convert_wind_speed(wind_kmh, wind_unit)
            return f"💨 Wind speed in {loc} {time} is {wind} {wind_unit}."
        return "Wind data unavailable."

#Example Use (for the final version) provided by Chatgbt.
locations = ["London", "FakeCity", "Sydney"]
weather = get_weather_data(locations)

for loc, result in weather.items():
    if "error" in result:
        print(f"❌ {loc}: {result['error']}")
    else:
        print(f"✅ {loc} — Current: {result['current_conditions']}")
        print("Forecast:")
        for day in result['forecast']:
            print(day)

